<a href="https://colab.research.google.com/github/ericyangyu/2020-25-Days-of-Serverless/blob/master/HW%203/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 158 HW 3

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/CSE 158/HW 3/')

In [ ]:
%cd /content/gdrive/My Drive/CSE\ 158/HW\ 3/

/content/gdrive/My Drive/CSE 158/HW 3


## Preprocessing Step 0

In [ ]:
import gzip
import random
import copy
import numpy as np
from collections import defaultdict

In [ ]:
# Extract JSON file
def readJSON(path):
    for l in gzip.open(path, 'rt'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u,g,d

In [ ]:
# Extract data as (userId, game id, 1 if played else 0)
data = [(u, g, 1) for u, g, _ in readJSON('train.json.gz')]

In [ ]:
# Perform train/val split
train_data = data[:165000]
val_data = data[165000:]

## Problem 1: Evaluate performance of baseline model on validation set based on user/item pairs that weren't played

In [ ]:
# Map users to game ids for all data (train and val)
u_g_map = defaultdict(list)
games_set = set() # Set of unique game ids
for u, g, _ in data:
    u_g_map[u].append(g)
    games_set.add(g)

In [ ]:
# Randomly generate 10k negative entries for each user in val set
neg_entries = [] # Keep negative entries for val set in here for now
for u, _, _ in val_data:
    # Keep sampling a game id for user u until we confirm negative
    neg_g = random.sample(games_set, 1)[0]
    while neg_g in u_g_map[u]:
        neg_g = random.sample(games_set, 1)[0]
        
    # Add new negative entry
    neg_entries.append((u, neg_g, 0))
    u_g_map[u].append(neg_g)
    
# Append negative entries to val data
val_data += neg_entries

In [ ]:
# Calculate acc between file with predicted labels and expected labels as a list
def calc_acc(pred_file, targets):
    with open(pred_file, 'r') as f:
        correct = 0
        total = 0
        for i, l in enumerate(f):
            if i == 0:
                continue
            _, pred = l.split(',')
            
            correct += int(int(pred) == targets[i-1][2])
            total += 1

    acc = correct/total
    return acc

In [ ]:
# Predict play based on whether game is considered popular
def baseline_q1(train_data, val_data):
    gameCount = defaultdict(int)
    totalPlayed = 0

    for i, (user, game, _) in enumerate(train_data):
        gameCount[game] += 1
        totalPlayed += 1

    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed/2: break

    predictions = open('q1.txt', 'w+')
    predictions.write('userID-gameID,prediction\n')
    for l in [u + '-' + g for u, g, _ in val_data]:
        if l.startswith("userID"):
            #header
            predictions.write(l)
            continue
        u,g = l.strip().split('-')
        if g in return1:
            predictions.write(u + '-' + g + ",1\n")
        else:
            predictions.write(u + '-' + g + ",0\n")

    predictions.close()

In [ ]:
# Run baseline predictions on val set
baseline_q1(train_data, val_data)

In [ ]:
val_acc = calc_acc('q1.txt', val_data)
print(f'Validation accuracy: {val_acc * 100}%')

Validation accuracy: 68.325%


The validation accuracy of this baselines can be seen above.

## Problem 2: Find a better threshold for play predictions

In [ ]:
thresh = 1.45 # The threshold I found through manual grid search

In [ ]:
# Redefine play prediction baselines with better threshold
# Predict play based on whether game is considered popular

def baseline_q2(train_data, val_data):
    gameCount = defaultdict(int)
    totalPlayed = 0

    for i, (user, game, _) in enumerate(train_data):
        gameCount[game] += 1
        totalPlayed += 1

    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed/thresh: break

    predictions = open('q2.txt', 'w+')
    predictions.write('userID-gameID,prediction\n')
    for l in [u + '-' + g for u, g, _ in val_data]:
        if l.startswith("userID"):
            #header
            predictions.write(l)
            continue
        u,g = l.strip().split('-')
        if g in return1:
            predictions.write(u + '-' + g + ",1\n")
        else:
            predictions.write(u + '-' + g + ",0\n")

    predictions.close()

In [ ]:
# Run baseline predictions on val set
baseline_q2(train_data, val_data)

In [ ]:
tmp = np.array(val_data)
labels = tmp.T[2]
labels = np.array(labels, dtype=np.int32)

In [ ]:
val_acc = calc_acc('q2.txt', val_data)
print(f'Validation accuracy: {val_acc * 100}%')

Validation accuracy: 70.26%


I played around with the threshold and found that dividing "totalPlayed" by 1.45 yields the highest test accuracy with the above shown. The table shown below shows my grid search.

#### Thresh: Val Acc
1.4: 69.905% <br>
1.45: 70.17999999999999% <br>
1.5: 70.13000000000001% <br>
1.55: 70.145% <br>
1.6: 70.03% <br>
1.65: 69.915% <br>

### Run a sanity check on Q2 (i.e. predict on pairs_Played.txt and submit to Kaggle)

In [ ]:
thresh = 1.45 # The threshold I found through manual grid search

In [ ]:
# Run the sanity check for Q2 by predicting on "pairs_Played.txt"
def sanity_check_q2(train_data, val_data):
    gameCount = defaultdict(int)
    totalPlayed = 0

    for user, game, _ in train_data:
        gameCount[game] += 1
        totalPlayed += 1

    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed/thresh: break

    predictions = open('sanity_q2.txt', 'w+')
    predictions.write('userID-gameID,prediction\n')
    test_data = []
    with open('pairs_Played.txt', 'r') as f:
        for i, l in enumerate(f):
            if i == 0: continue
            u, g = l.strip().split('-')
            test_data.append((u,g))
    for l in [u + '-' + g for u, g in test_data]:
        if l.startswith("userID"):
            #header
            predictions.write(l)
            continue
        u,g = l.strip().split('-')
        if g in return1:
            predictions.write(u + '-' + g + ",1\n")
        else:
            predictions.write(u + '-' + g + ",0\n")

    predictions.close()

Kaggle accuracy for sanity_q2.txt: 70.300%

Note: this is not my answer for Q2, this is just me reporting my result for performing the recommended sanity check as per https://piazza.com/class/kfmxzxpkrfu2db?cid=387. The answer for Q2 is further above.

## Problem 3: Stronger baselines using Jaccard similarity

In [ ]:
# Define Jaccard similarity 
def jaccard_sim(s1, s2):
    if type(s1) == list:
        s1 = set(s1)
    if type(s2) == list:
        s2 = set(s2)
    num = len(s1.intersection(s2))
    den = len(s1.union(s2))
    return num / den

In [ ]:
# Calculate max jaccard similarity of g given mapping of users to game ids and game ids to users
def max_jaccard(u, g, g_u_map, u_g_map):
    max_sim = 0
    for g_prime in u_g_map[u]:
        if g_prime == g: 
            continue
        # Update max jaccard similarity we've seen so far
        max_sim = max(max_sim, jaccard_sim(g_u_map[g], g_u_map[g_prime]))
        
    return max_sim

In [ ]:
# Map users to game ids (for train and val)
g_u_map = defaultdict(list)
for u, g, _ in train_data:
    g_u_map[g].append(u)
    
# Map game ids to users for all data (train and val)
u_g_map = defaultdict(list)
for u, g, _ in train_data:
    u_g_map[u].append(g)

In [ ]:
# For each (u, g, _), predict 'played' if the max of the jaccard similarities between g and g' where g' is game ∈ {user games played}  - {g}
def baseline_q3(val_data, thresh):
    with open('q3.txt', 'w+') as f:
        f.write('userID-gameID,prediction\n')
        for i, (u, g, _) in enumerate(val_data):
            max_sim = max_jaccard(u, g, g_u_map, u_g_map)
            # Perform prediction based on threshold
            f.write(u + '-' + g + ',' + str(int(max_sim > thresh))) # Store the pred
            f.write('\n')

In [ ]:
# Calculate our predictions for q3
jaccard_thresh = 0.04 # Use manually grid searched best thresh
baseline_q3(val_data, jaccard_thresh)

In [ ]:
# Calculate val acc of baseline predictions on val set
with open('q3.txt', 'r') as f:
    correct = 0
    total = 0
    for i, l in enumerate(f):
        if i == 0:
            continue
        _, pred = l.strip().split(',')
       
        correct += int(int(pred) == val_data[i-1][2])
        total += 1
        

In [ ]:
val_acc = calc_acc('q3.txt', val_data)
print(f'Validation accuracy: {correct/total * 100}%')

Validation accuracy: 64.505%


## Problem 4: Implement Jaccard-Similarity and Popularity based 

In [ ]:
popularity_thresh = 1.45 # The threshold I found for popularity through manual grid search
jaccard_thresh = 0.01 # The threshold I found for jaccard to work best

In [ ]:
def baseline_q4(train_data, val_data, g_u_map, u_g_map, file_name='q4.txt'):

    gameCount = defaultdict(int)
    totalPlayed = 0

    for i, (user, game, _) in enumerate(train_data):
        gameCount[game] += 1
        totalPlayed += 1

    mostPopular = [(gameCount[x], x) for x in gameCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPlayed/popularity_thresh: break

    predictions = open(file_name, 'w+')
    for l in [u + '-' + g for u, g, _ in val_data]:
        if l.startswith("userID"):
            #header
            predictions.write(l + '\n')
            continue
        u,g = l.strip().split('-')
        
        # Do max jaccard similarity here
        max_sim = max_jaccard(u, g, g_u_map, u_g_map)
            
        if g in return1 and max_sim > jaccard_thresh:
            predictions.write(u + '-' + g + ",1\n")
        else:
            predictions.write(u + '-' + g + ",0\n")

    predictions.close()

In [ ]:
baseline_q4(train_data, val_data, g_u_map, u_g_map)

In [ ]:
# Calculate validation accuracy for Q4
val_acc = calc_acc('q4.txt', val_data)
print(f'Validation accuracy: {val_acc * 100}%')

Validation accuracy: 70.14850742537126%


## Problem 5: Predict on pairs_Played.txt and submit to kaggle

In [ ]:
# Extract out pairs_Played data
pairs_data = []
with open('pairs_Played.txt', 'r') as f:
    for l in f:
        u, g = l.strip().split('-')
        pairs_data.append((u, g, 1))

In [ ]:
baseline_q4(train_data, pairs_data, g_u_map, u_g_map, 'q5.txt')

Kaggle username: eyyu12321 <br>
Kaggle Accuracy: 70.550%

NOTE: on the leaderboards, my name shows up as Eric Yu.

## Problem 6

In [ ]:
import gzip
import re
from sklearn import linear_model
from collections import Counter

In [ ]:
def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [ ]:
# Given a string, extract words and remove newlines, punctuation, and upper cases
def extract_words(text):
    text = text.replace('\n', ' ') # remove newlines
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    text = text.lower() # Lower case everything
    text = [word.strip() for word in text.split(' ')] # Extract every word
    return text

In [ ]:
# Load in category data
data = list(parse("train_Category.json.gz"))

In [ ]:
data[0]

{'date': '2014-02-07',
 'early_access': False,
 'genre': 'Adventure',
 'genreID': 3,
 'hours': 4.1,
 'reviewID': 'r75487422',
 'text': 'Short Review:\nA good starting chapter for this series, despite the main character being annoying (for now) and a short length. The story is good and actually gets more interesting. Worth the try.\nLong Review:\nBlackwell Legacy is the first on the series of (supposedly) 5 games that talks about the main protagonist, Rosangela Blackwell, as being a so called Medium, and in this first chapter we get to know how her story will start and how she will meet her adventure companion Joey...and really, that\'s really all for for now and that\'s not a bad thing, because in a way this game wants to show how hard her new job is, and that she cannot escape her destiny as a medium.\nMy biggest complain for this chapter, except the short length, it\'s the main protagonist being a "bit" too annoying to be likeable, and most of her dialogues will always be about compl

In [ ]:
# Perform train val split
train_data = data[:165000]
val_data = data[165000:]

In [ ]:
# Remove punctuation and capitalization and find 1000 most common words
# First, extract all text into single vector
text_data = []
for datum in train_data:
    text = datum['text']
    text = extract_words(text)
    text_data += text

In [ ]:
# Count frequencies of each word
text_freq_map = Counter(text_data)

# Keep track of top 1k words
onek_most_common_words = [(word, freq) for freq, word in sorted([(val, key) for key, val in text_freq_map.items()], reverse=True)[:1000]]

In [ ]:
# Report top 10 most common words and frequencies
for i, (word, freq) in enumerate(onek_most_common_words[:10]):
    print(f'#{i+1} most frequent word: {word}')
    print(f'frequency: {freq}')

#1 most frequent word: the
frequency: 544773
#2 most frequent word: and
frequency: 317702
#3 most frequent word: a
frequency: 305503
#4 most frequent word: to
frequency: 291915
#5 most frequent word: game
frequency: 245512
#6 most frequent word: of
frequency: 227426
#7 most frequent word: is
frequency: 208533
#8 most frequent word: you
frequency: 200730
#9 most frequent word: i
frequency: 195982
#10 most frequent word: it
frequency: 191005


## Problem 7

In [ ]:
import pickle

In [ ]:
def get_x_y(dataset, most_common_words):
    # Create 1k dimensional feature vector representing frequencies of each top 1k words that appear
    # in a review
    x = np.zeros((len(dataset), len(most_common_words)))
    y = np.zeros(len(dataset))

    # Create a map indexing each top freq word for mapping to feature vector
    word_idx_map = {}
    for i, (word, _) in enumerate(most_common_words):
        word_idx_map[word] = i
    
    # Populate X and y
    for row, datum in enumerate(dataset):
        # populate X first
        text = datum['text'] 
        text = extract_words(text) # extract words
        word_freq_map = Counter(text) # Count up frequencies of each unique word in current review text
        # Map frequencies to corresponding row and top word index
        for word, freq in word_freq_map.items():
            idx = word_idx_map.get(word)
            if idx == None: 
                continue
            x[row][idx] = freq
    
        # Now populate y
        y[row] = int(datum['genreID'])

    return x, y

In [ ]:
# Loads up a model from pickle
def load_model(filename):
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    return model

In [ ]:
# Populate train data
x_train, y_train = get_x_y(train_data, onek_most_common_words)

In [ ]:
# Do what we did above, but this time for validation x and y
x_val, y_val = get_x_y(val_data, onek_most_common_words)

In [ ]:
# Build our logistic regressor and perform multinomial classification
def log_reg(x, y, x_test, y_test, solver='lbfgs', C=1, max_iter=100, savefile='log_reg_model.pkl'):
    # Define log reg model
    model = linear_model.LogisticRegression(solver=solver, fit_intercept=True, max_iter=max_iter, C=C)

    # Fit model
    model.fit(x, y)

    # Save our model
    with open(savefile, 'wb') as f:
        pickle.dump(model, f)

    # Predict on test set passed in, which can also be validation set
    preds = model.predict(x_test)

    # Calculate test accuracy
    preds, y_test = np.array(preds), np.array(y_test)
    correct = np.sum((preds == y_test).astype(np.int32))
    total = preds.shape[0]
    acc = correct / total
    return acc


In [ ]:
print(x_train.shape)

(165000, 1000)


In [ ]:
# Calculate validation performance
# NOTE: we use smaller train size to save time
n = 20000
val_acc = log_reg(x_train[:n], y_train[:n], x_val, y_val, max_iter=100, savefile='q7_log_reg_model.pkl')
print(f'Validation accuracy: {val_acc * 100}%')

Validation accuracy: 66.42%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### Validation Accuracy of our model can be seen right above this box.

## Problem 8: Improve performance of classifier

In [ ]:
# Increase dictionary size by 500, and include regularization of 0.01

# Keep track of top 3000 words
q8_most_common_words = [(word, freq) for freq, word in sorted([(val, key) for key, val in text_freq_map.items()], reverse=True)[:2000]]

In [ ]:
# Populate train data
x_train, y_train = get_x_y(train_data, q8_most_common_words)

# Do what we did above, but this time for validation x and y
x_val, y_val = get_x_y(val_data, q8_most_common_words)

In [ ]:
# Initialize c
C = 0.1

# Train with new regularization constant
n = 100000
val_acc = log_reg(x_train[:n], y_train[:n], x_val, y_val, max_iter=500, C=C, savefile='q8_log_reg_model.pkl')

print(f'Validation accuracy: {val_acc * 100}%')

Validation accuracy: 69.42%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### Validation Accuracy of our new model can be seen right above this box. Note that it performs better than Q7 model.

In [ ]:
def get_x_pairs_category(urs, ur_t_map, most_common_words):
    # Create 1k dimensional feature vector representing frequencies of each top 1k words that appear
    # in a review
    x = np.zeros((len(urs), len(most_common_words)))

    # Create a map indexing each top freq word for mapping to feature vector
    word_idx_map = {}
    for i, (word, _) in enumerate(most_common_words):
        word_idx_map[word] = i
    
    # Populate X and y
    for row, (user_id, review_id) in enumerate(urs):
        # populate X first
        text = ur_t_map[(user_id, review_id)]
        word_freq_map = Counter(text) # Count up frequencies of each unique word in current review text
        # Map frequencies to corresponding row and top word index
        for word, freq in word_freq_map.items():
            idx = word_idx_map.get(word)
            if idx == None: 
                continue
            x[row][idx] = freq

    return x

In [ ]:
def predict_sanity(model, ur_t_map, most_common_words, filename, savefile):
    # Predict on pairs_Category to send to kaggle
    urs = []
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            if i == 0: continue
            u, r = l.split('-')
            u, r = u.strip(), r.strip()
            urs.append((u, r))

    # Extrapolate x
    x = get_x_pairs_category(urs, ur_t_map, most_common_words)

    # Compute predicted values
    preds = model.predict(x)

    # Zip up into file and save
    with open(savefile, 'w+') as f:
        f.write('userID-reviewID,prediction\n')
        for (u, r), pred in zip(urs, preds):
            f.write(u + '-' + r.strip() + ',' + str(int(pred)) + '\n')

In [ ]:
# Load in test category data
test_data = list(parse("test_Category.json.gz"))

In [ ]:
# Construct a user to text mapping
ur_t_map = defaultdict(list)
for datum in test_data:
    user_id = datum['userID']
    review_id = datum['reviewID']
    text = extract_words(datum['text'])
    ur_t_map[(user_id, review_id)] += text

In [ ]:
# Load up our trained model
model = load_model('q8_log_reg_model.pkl')

In [ ]:
# Predict on pairs_Category.txt and it will save into q8.txt. We will then submit q8.txt to kaggle
predict_sanity(model, ur_t_map, q8_most_common_words, 'pairs_Category.txt', 'q8.txt')

### Kaggle Category Score

Accuracy: 69.480%